In [ ]:
from sentence_transformers import SentenceTransformer

# Download and save the model
model = SentenceTransformer("myrkur/sentence-transformer-parsbert-fa-2.0")
model.save("E_model/models/parsbert-fa-2.0") 

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/70.2k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/646 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.22M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.81M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [52]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(model_name="app/E_model/models/parsbert-fa-2.0")
vectorstore = FAISS.load_local("app/faiss_index_deduplicated", embedding_model,allow_dangerous_deserialization=True)
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 7})

#Needs Normalization
# embedding_model = HuggingFaceEmbeddings(model_name="app/E_model/models/parsbert-fa-2.0")
# vectorstore = FAISS.load_local("app/faiss_index_deduplicated", embedding_model,allow_dangerous_deserialization=True)
# retriever = vectorstore.as_retriever(search_type="similarity_score_threshold", search_kwargs={
#         "k": 10,                  
#         "score_threshold": 0.7
#     })


In [48]:
retriever.invoke("نقش پیش‌زمینه ژنتیکی در بروز میگرن را ارزیابی کنید")

[Document(id='97d99a85-662c-4911-87b4-0d1eff80f8aa', metadata={'title': 'عامل ژنتیکی'}, page_content='مطالعه دوقلوها نشان می\u200cدهد که تأثیر ژنتیکی ۳۴ تا ۵۱ ٪ در احتمال ابتلا به میگرن مؤثر می\u200cباشد. این رابطه ژنتیکی در میگرن\u200cهایی که دارای آئورا هستند نسبت به میگرن\u200cهایی که آئورا ندارند قوی\u200cتر می\u200cباشد. برخی از گونه\u200cهای ویژه ژن\u200cها، خطر ابتلا به این بیماری را به میزان کم تا متوسط افزایش می\u200cدهند. \nاختلالات ژنتیکی که باعث به وجود آمدن میگرن می\u200cشوند، اندک هستند. یکی از این گونه\u200cها به عنوان «میگرن نیم فلج خانوادگی» شناخته شده\u200cاست، که نوعی میگرن با آئورا می\u200cباشد، و از طریق کروموزوم غیرجنسی غالب به ارث می\u200cرسد. \nاین اختلالات با انواع کدگذاری ژنی برای پروتئین\u200cهایی که کار انتقال یون را بر عهده دارند، در ارتباط می\u200cباشند.'),
 Document(id='ec050a6b-73f0-4b30-bb45-52e86301ae4b', metadata={'title': 'بیماریزایی'}, page_content='ممکن است بیمار زمینه ارثی داشته باشد، بروز هم\u200cزمان این مشکل در دوقلوهای تک تخمکی بین ۹۰ – ۳۳ ٪ م

In [2]:
# from langchain_core.runnables import RunnablePassthrough
# from langchain_core.prompts import PromptTemplate
# from langchain_ollama.llms import OllamaLLM

# def format_docs(docs):
#     return "\n\n".join(doc.page_content for doc in docs)

# template = """Use the following pieces of context to answer the question at the end in Persian.
# If you don't know the answer, just say that you don't know, don't try to make up an answer.
# keep the answer as concise as possible.


# {context}

# Question: {question}

# Helpful Answer:"""

# model = OllamaLLM(model="aya-expanse")

# custom_rag_prompt = PromptTemplate.from_template(template)

# rag_chain = (
#     {"context": retriever | format_docs, "question": RunnablePassthrough()}
#     | custom_rag_prompt
#     | model
# )
#print(rag_chain.invoke("ابسه در کجای بدن میتواند شکل بگیرد؟"))

In [49]:
#RAG Chain With HyDE
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_ollama.llms import OllamaLLM
from langchain_core.output_parsers import StrOutputParser

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# 1. HyDE Generation Components
hyde_template = """Write a hypothetical answer in Persian to the question below. 
This answer will help retrieve relevant documents. Even if unsure, provide a plausible example.

Question: {question}

Hypothetical Answer:"""
hyde_prompt = PromptTemplate.from_template(hyde_template)

model = OllamaLLM(model="aya-expanse")

# Chain to generate hypothetical answer

hyde_generator = (
    {"question": RunnablePassthrough()} 
    | hyde_prompt 
    | model 
    | StrOutputParser() 
)


retriever_with_hyde = (
    hyde_generator  
    | retriever  
    | format_docs  
)


template = """Use the following pieces of context to answer the question at the end in Persian.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
keep the answer as concise as possible.


{context}

Question: {question}

Helpful Answer:"""
custom_rag_prompt = PromptTemplate.from_template(template)

rag_chain = (
    {"context": retriever_with_hyde, "question": RunnablePassthrough()} 
    | custom_rag_prompt 
    | model
)

In [60]:
import gradio as gr
def generate_answer(question):
    retrieval_result = retriever.invoke(question)  # Retrieve relevant docs
    formatted_docs = format_docs(retrieval_result)  # Format retrieved chunks
    answer = rag_chain.invoke(question)  # Get the final answer

    return formatted_docs, answer

# Gradio UI
iface = gr.Interface(
    fn=generate_answer,
    inputs=gr.Textbox(label="Type Your Question", placeholder="Enter your question here..."),
    outputs=[
        gr.Markdown(label="Retrieved Context",container=True),  # Show retrieved documents
        gr.Textbox(label="Answer")  # Show the generated response
    ],
    title="RAG Interface",
    description="Healthcare RAG Pipeline with Disease Information"
)

# Launch the Gradio app
iface.launch(share=True)

* Running on local URL:  http://127.0.0.1:7861

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


In [75]:
iface.close()

Closing server running on port: 7861


Testing The Model

In [53]:
import pandas as pd

df = pd.read_csv('Final_Questions.csv')
df

,Q,GT,A
0,ویروس مسئول ایجاد بیماری کووید-۱۹ چیست؟,SARS-CoV-2,NaN
1,کدام عضو بدن بیشتر تحت تأثیر هپاتیت قرار می‌گیرد؟,کبد,NaN
2,اصطلاح رایج برای فشار خون بالا چیست؟,فشار خون بالا,NaN
3,کدام بیماری با سطح بالای قند خون مشخص می‌شود؟,دیابت ملیتوس,NaN
4,راه اصلی انتقال سرماخوردگی چیست؟,قطرات هوابرد,NaN
5,کدام باکتری شایع‌ترین عامل ایجاد سل است؟,مایکوباکتریوم توبرکلوزیس,NaN
6,کدام اختلال ژنتیکی با وجود یک کروموزوم ۲۱ اضاف...,سندرم داون,NaN
7,کدام بیماری خودایمنی منجر به تخریب سلول‌های بت...,دیابت نوع ۱,NaN
8,ناقل اصلی انتقال مالاریا به انسان چیست؟,پشه‌های آنوفل,NaN
9,کدام بیماری نورودژنراتیو با تجمع پلاک‌های آمیل...,بیماری آلزایمر,NaN


In [54]:
def RAG(question):
    answer = rag_chain.invoke(question)  # Get the final answer
    return answer

df ['A'] = df ['Q'].apply(RAG)
df

,Q,GT,A
0,ویروس مسئول ایجاد بیماری کووید-۱۹ چیست؟,SARS-CoV-2,ویروس مسئول ایجاد بیماری کووید-۱۹، یک **کروناو...
1,کدام عضو بدن بیشتر تحت تأثیر هپاتیت قرار می‌گیرد؟,کبد,کبد (لبر) بیشتر تحت تأثیر هپاتیت قرار می‌گیرد....
2,اصطلاح رایج برای فشار خون بالا چیست؟,فشار خون بالا,اصطلاح رایج برای فشار خون بالا، **پرفشاری خون*...
3,کدام بیماری با سطح بالای قند خون مشخص می‌شود؟,دیابت ملیتوس,دیابت با سطح بالای قند خون مشخص می‌شود. به طور...
4,راه اصلی انتقال سرماخوردگی چیست؟,قطرات هوابرد,سرماخوردگی عمدتاً از طریق تماس مستقیم با فرد م...
5,کدام باکتری شایع‌ترین عامل ایجاد سل است؟,مایکوباکتریوم توبرکلوزیس,شایع‌ترین عامل ایجاد سل، باکتری *Mycobacterium...
6,کدام اختلال ژنتیکی با وجود یک کروموزوم ۲۱ اضاف...,سندرم داون,ترویج مغزی (Down Syndrome) یا تریزومی ۲۱، یک ا...
7,کدام بیماری خودایمنی منجر به تخریب سلول‌های بت...,دیابت نوع ۱,دیابت نوع ۱ ناشی از خودایمنی است که منجر به تخ...
8,ناقل اصلی انتقال مالاریا به انسان چیست؟,پشه‌های آنوفل,ناقل اصلی انتقال مالاریا به انسان پشه آنوفل جن...
9,کدام بیماری نورودژنراتیو با تجمع پلاک‌های آمیل...,بیماری آلزایمر,بیماری نورودژنراتیو که با تجمع پلاک‌های آمیلوئ...


In [63]:
def Retrived_Docs(question):
    r = retriever_with_hyde.invoke(question)
    return r
df ['Retrieved Docs'] = df ['Q'].apply(Retrived_Docs)
df

,Q,GT,A,Retrieved Docs
0,ویروس مسئول ایجاد بیماری کووید-۱۹ چیست؟,SARS-CoV-2,ویروس مسئول ایجاد بیماری کووید-۱۹، یک **کروناو...,ویروس کرونا (نام علمی: Coronaviruses) یا ویروس...
1,کدام عضو بدن بیشتر تحت تأثیر هپاتیت قرار می‌گیرد؟,کبد,کبد (لبر) بیشتر تحت تأثیر هپاتیت قرار می‌گیرد....,نارسایی کبد به معنی کاهش شدید عملکرد کبد است. ...
2,اصطلاح رایج برای فشار خون بالا چیست؟,"پرفشاری شریانی, Hypertension",اصطلاح رایج برای فشار خون بالا، **پرفشاری خون*...,فشار خون بالا یا پرفشاری خون (به انگلیسی: Hype...
3,کدام بیماری با سطح بالای قند خون مشخص می‌شود؟,دیابت ملیتوس,دیابت با سطح بالای قند خون مشخص می‌شود. به طور...,دیابت شیرین یا بیماری قند (به انگلیسی: Diabete...
4,راه اصلی انتقال سرماخوردگی چیست؟,قطرات هوابرد,سرماخوردگی عمدتاً از طریق تماس مستقیم با فرد م...,ویروس سرماخوردگی اغلب به دو شکل منتقل می‌شود، ...
5,کدام باکتری شایع‌ترین عامل ایجاد سل است؟,مایکوباکتریوم توبرکلوزیس,شایع‌ترین عامل ایجاد سل، باکتری *Mycobacterium...,سل (به انگلیسی: Tuberculosis یا TB)، یک بیماری...
6,کدام اختلال ژنتیکی با وجود یک کروموزوم ۲۱ اضاف...,سندرم داون,ترویج مغزی (Down Syndrome) یا تریزومی ۲۱، یک ا...,آپلازی مادرزادی خالص گلبول قرمز (Congenital Pu...
7,کدام بیماری خودایمنی منجر به تخریب سلول‌های بت...,دیابت نوع ۱,دیابت نوع ۱ ناشی از خودایمنی است که منجر به تخ...,دیابت ناشی از واکنش ایمنی، (Type ۱ A) یک اختلا...
8,ناقل اصلی انتقال مالاریا به انسان چیست؟,پشه‌های آنوفل,ناقل اصلی انتقال مالاریا به انسان پشه آنوفل جن...,مالاریا (به انگلیسی: Malaria) یک بیماری انگلی ...
9,کدام بیماری نورودژنراتیو با تجمع پلاک‌های آمیل...,بیماری آلزایمر,بیماری نورودژنراتیو که با تجمع پلاک‌های آمیلوئ...,بیماری آلزایمر هنگامی رخ می‌دهد که مقادیر غیرط...


In [56]:
df.to_csv('Final_Questions_Answers.csv', index=False, encoding='utf-8-sig')

In [70]:
for i in range(24):
    print(f"""
    Question : {df["Q"][i]}\n
    GT :  {df['GT'][i]}\n
    Answer: {df['A'][i]}
    """)


    Question : ویروس مسئول ایجاد بیماری کووید-۱۹ چیست؟

    GT :  SARS-CoV-2

    Answer: ویروس مسئول ایجاد بیماری کووید-۱۹، یک **کروناویروس** به نام **SARS-CoV-2** (ویروس سرفه شدید تنفسی کرونای نوع 2) است. این ویروس از خانواده کروناویروس‌ها می‌باشد که همچنین شامل ویروس‌های عامل سرماخوردگی معمولی، سارس و مرس است.

SARS-CoV-2 دارای یک ژنوم RNA تک رشته‌ای مثبت است و به سلول‌های میزبان خود با اتصال پروتئین اسپیک (Spike) موجود بر روی غشاء ویروس به گیرنده ACE2 در سلول‌های میزبان انسان نفوذ می‌کند. پس از ورود به سلول، ویروس ژنوم خود را وارد هسته سلول می‌کند و با استفاده از ماشین‌آلات سلولی میزبان، نسخه‌های بیشتری از خود را تولید می‌نماید. این فرایند می‌تواند منجر به عفونت سیستمیک و علائمی مانند سرفه، تب، خستگی و در موارد شدیدتر، پنومونیا شود.
    

    Question : کدام عضو بدن بیشتر تحت تأثیر هپاتیت قرار می‌گیرد؟

    GT :  کبد

    Answer: کبد (لبر) بیشتر تحت تأثیر هپاتیت قرار می‌گیرد. هپاتیت یک بیماری ویروسی است که به طور مستقیم بر روی کبد تأثیر می‌گذارد و می‌تواند منجر به التهاب، نارسایی 

In [74]:
print(df['A'][11])

اختلالی که شما توصیف کردید، به نظر می‌رسد به **کریسول (Photodamage)** مربوط باشد. این یک اختلال ژنتیکی نادر است که در آن بدن قادر به ترمیم صحیح آسیب‌های DNA ناشی از نور ماوراءبنفش (UV) نیست.

در کریسول، ژن‌هایی که مسئول تولید پروتئین‌های تعمیر کننده DNA هستند، دچار جهش می‌شوند. این امر باعث می‌شود سلول‌ها نتوانند به درستی آسیب‌های ایجاد شده توسط اشعه UV را ترمیم کنند، که منجر به پیری زودرس پوست و افزایش خطر ابتلا به سرطان پوست می‌شود.

این وضعیت را می‌توان با روش‌های مختلفی مدیریت کرد، از جمله:

- اجتناب از قرار گرفتن در معرض نور خورشید و استفاده از کرم‌های ضدآفتاب قدرتمند.
- درمان‌های پزشکی برای کاهش علائم و عوارض.
- مراقبت‌های منظم پوستی برای حفظ سلامت پوست.

درمان ژنتیکی نیز در حال بررسی است تا مکانیسم‌های ترمیم DNA را هدف قرار دهد و به این ترتیب، کیفیت زندگی افراد مبتلا به این اختلال را بهبود بخشد.


In [68]:
print(df['Q'][11])

کدام اختلال ژنتیکی نادر با ناتوانی بدن در ترمیم آسیب‌های DNA ناشی از نور ماوراءبنفش مشخص می‌شود؟


In [69]:
retrieval_result = retriever.invoke("کدام اختلال ژنتیکی نادر با ناتوانی بدن در ترمیم آسیب‌های DNA ناشی از نور ماوراءبنفش مشخص می‌شود؟")  # Retrieve relevant docs
print(format_docs(retrieval_result))

جهش اتوزومال غالب ژن ACVR ۱ یا activin-like kinase ۲ [ALK- ۲] در کروموزوم ۲ q ۲۳ - ۲۴ عامل بیماری است. 
استخوانی شدن گاه خودبخود یا بدنبال تروماهای خفیف ایجاد می‌شود. پیشرفت آن گاه موجب جمود دائمی مفاصل می‌شود. از علائم بیماری کوتاه بودن انگشت شست پا و هالوکس والگوس (انحراف شست به داخل) است. در مراحل پایانی شخص به مجسمه‌ای انسان‌نما تبدیل می‌شود که امکان خم شدن ندارد و دقیقا مانند یک مجسمه او را جابه‌جا می‌کنند. گاه جراحی ضرورت می‌یابد.

WHO در حال حاضر مهندسی ژنتیک ویروس واریولا را ممنوع کرده است. با این حال، در سال ۲۰۰۴، کمیته مشورتی WHO به اجازه ویرایش ژنوم دو نمونه باقی‌مانده از ویروس واریس ماژور برای افزودن یک ژن نشانگر رای داد. این ژن که GFP یا پروتئین فلورسنت سبز نامیده می‌شود، باعث می‌شود نمونه‌های زنده ویروس در زیر نور فلورسنت سبز بدرخشند. قرار دادن این ژن، که بر حدت ویروس تأثیر نمی‌گذارد، تنها تغییر مجاز ژنوم خواهد بود. کمیته بیان کرد که اصلاح پیشنهادی با آسان‌تر کردن ارزیابی اینکه آیا یک درمان بالقوه در از بین بردن نمونه‌های ویروسی مؤثر است یا خیر، به تحقیق در مورد درمان‌ها 